In [2]:
import numpy as np

# Résolution de Ax=b avec A une matrice carrée n×n et b∈Rn

## Résolution d'un système linéaire triangulaire

In [16]:
def solve_upper(A, b):
    assert A.shape[0] == b.shape[0]
    n = b.shape[0]
    x= np.zeros((n,))
    x[n-1] = b[n-1]/A[n-1, n-1]
    for i in range(n-1, 0-1, -1):
        x[i] = (b[i] - np.sum(A[i, i+1:]*x[i+1:]))/A[i, i]
    print(x)

A = np.random.randint(1, 10, (4, 4))
b = np.random.randint(1, 10, (4, ))

mask_l = np.fromfunction(lambda i, j : i>j, (4,4))
A[mask_l] = 0
print(A, b)
solve_upper(A, b)

[[5 6 5 1]
 [0 6 7 8]
 [0 0 1 2]
 [0 0 0 1]] [8 3 8 2]
[688. 103.  12.   2.]


In [15]:
A = np.random.randint(1, 10, (4, 4))
b = np.random.randint(1, 10, (4, ))

mask_l = np.fromfunction(lambda i, j : i>j, (4, 4))
A[mask_l] = 0
A

array([[1, 1, 5, 1],
       [0, 7, 4, 8],
       [0, 0, 1, 7],
       [0, 0, 0, 2]])

In [26]:
i = 1
A[i, i+1:], b[i+1:]

(array([4, 7]), array([2, 8]))